### **Data Load Proccess**:

**Objective**: This notebook shows the process of loading data into a sql database. For this, a free instance of [Render](https://github.com/DCajiao/workshop001_candidates_analysis/blob/main/docs/database/how_to_deploy_databases_on_render.md) will be used and the `merged_data` will be uploaded.

---

#### **First Step**: Load clean, processed and previously transformed data from a csv.

Task:
- Load the merge dataset. 
- Find insights with the functions of analysis_functions.py
- What data types do we have?
- What categorical variables do we have?

In [19]:
import pandas as pd
import sys
import os

# Add the 'src' folder to sys.path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# Remove the column display limit to show all columns in the DataFrame
pd.set_option('display.max_columns', None)

In [20]:
merged_data_df = pd.read_csv('../data/processed/merged_data.csv', encoding='utf-8')

In [21]:
print(f'The dataset has {merged_data_df.shape[0]} rows and {merged_data_df.shape[1]} columns')
print(f'The columns are: {merged_data_df.columns.tolist()}')

The dataset has 781 rows and 8 columns
The columns are: ['Happiness_Rank', 'Country', 'Happiness_Score', 'GDP_per_capita', 'Freedom', 'Generosity', 'Perceptions_of_corruption', 'Year']


---

#### **Second Step**: Write the queries for the PostgreSQL database based on the data.

Task:
- Create the query to create the schema adjusted to the data we have (`sql/transformed_data_schema.sql`).
- Create the query to upload the data seed we have (`sql/transformed_data_seed_data.sql`).

In [22]:
from utils.pysqlschema import SQLSchemaGenerator

In [23]:
generator = SQLSchemaGenerator(table_name='transformed_data')

generator.generate_schema(merged_data_df, '../sql/transformed_data_schema.sql')
generator.generate_seed_data(merged_data_df, '../sql/transformed_data_seed_data.sql')

INFO:root:Generating schema for transformed_data
INFO:root:Infering SQL type for int64
INFO:root:Infering SQL type for object
INFO:root:Infering SQL type for float64
INFO:root:Infering SQL type for float64
INFO:root:Infering SQL type for float64
INFO:root:Infering SQL type for float64
INFO:root:Infering SQL type for float64
INFO:root:Infering SQL type for int64
INFO:root:Query written to ../sql/transformed_data_schema.sql
INFO:root:Generating seed data for transformed_data
INFO:root:Query written to ../sql/transformed_data_seed_data.sql


---

#### **Third Step**: Upload data to database

Task:
- Import db class to use connector
- Establish connection and execute the queries to create the schema and send the data.
- Validate that the table has been created and that all records have been loaded.

In [24]:
# Instantiate the DB class
from connections.db import DB
db = DB()

INFO:root:Database credentials loaded successfully.


In [25]:
# Check the tables sizes
db.execute_query_file("../sql/queries/001_view_tables_sizes.sql", fetch_results=True)

INFO:root:✔ Connected to database
INFO:root:✔ Query executed
INFO:root:✔ Cursor closed
INFO:root:✔ Connection closed


[]

In [26]:
# Remove the table if it already exists
db.execute_query("DROP TABLE IF EXISTS transformed_data;", fetch_results=False)

INFO:root:✔ Connected to database
INFO:root:✔ Query executed
INFO:root:✔ Cursor closed
INFO:root:✔ Connection closed


In [27]:
# Create schema
db.execute_query_file("../sql/transformed_data_schema.sql", fetch_results=False)

INFO:root:✔ Connected to database
INFO:root:✔ Query executed
INFO:root:✔ Cursor closed
INFO:root:✔ Connection closed


In [29]:
# Seed data
db.execute_query_file("../sql/transformed_data_seed_data.sql", fetch_results=False)

INFO:root:✔ Connected to database
INFO:root:✔ Query executed
INFO:root:✔ Cursor closed
INFO:root:✔ Connection closed


In [30]:
# Check the tables sizes
db.execute_query_file("../sql/queries/001_view_tables_sizes.sql", fetch_results=True)

INFO:root:✔ Connected to database
INFO:root:✔ Query executed
INFO:root:✔ Cursor closed
INFO:root:✔ Connection closed


[('public.transformed_data', 781)]

---